In [1]:
import tensorflow as tf

In [2]:
a, b, c = [tf.Variable(i, name = name) for i, name in enumerate(["a", "b", "c"])]

In [3]:
init = tf.global_variables_initializer()

### Operations we want to execute together 

We want to switch values of `a` and `b`. We do it by first assigning `a` to `c` to remember it then assigning `b` to `a` and finally `c` to `a`.

In [4]:
op0 = tf.assign(c, a, name = "op0")
op1 = tf.assign(a, b, name = "op1")
op2 = tf.assign(b, c, name = "op2")

In [6]:
with tf.Session() as sess:
    sess.run(init)
    sess.run(op0)
    sess.run(op1)
    sess.run(op2)
    print(sess.run([a, b, c]))

[1, 0, 0]


We would like to group this into one operation.

### Grouping using `tf.group`

In [9]:
group = tf.group(
    tf.assign(c, a, name = "op0"),
    tf.assign(a, b, name = "op1"),
    tf.assign(b, c, name = "op2"),
    name = "group"
)

Unfortunately the order of execution is not guaranted:

In [10]:
with tf.Session() as sess:
    sess.run(init)
    sess.run(group)
    print(sess.run([a, b, c]))

[2, 2, 0]


### Controlling the order with `tf.control_dependencies`

In [11]:
with tf.control_dependencies([op0]):
    op0_op1 = tf.assign(a, b, name = "op0_op1")

In [14]:
with tf.control_dependencies([op0_op1]):
    op0_op1_op2 = tf.assign(b, c, name = "op0_op1_op2")

In [15]:
with tf.Session() as sess:
    sess.run(init)
    sess.run(op0_op1_op2)
    print(sess.run([a, b, c]))

[1, 0, 0]


# WARNING: Strange behaviour

In [4]:
xxx = [tf.Variable(i, name = "x{}".format(i)) for i in range(5)]
init = tf.global_variables_initializer()

In [7]:
steps = [
    tf.assign(b, a, name = "shift_{}".format(i))
    for i, (a, b) in enumerate(zip(xxx[:-1], xxx[1:]))
]

In [10]:
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(xxx))
    for s in steps:
        sess.run(s)
        print(sess.run(xxx)) 

[0, 1, 2, 3, 4]
[0, 0, 2, 3, 4]
[0, 0, 0, 3, 4]
[0, 0, 0, 0, 4]
[0, 0, 0, 0, 0]


## This surprisingly does not work

In [144]:
cum_steps = [steps[0]]
for i, s in enumerate(steps[1:]):
    with tf.control_dependencies([cum_steps[-1]]):
        cum_steps.append(tf.identity(s, "cs{}".format(i + 1)))

In [160]:
## Run it several times: the behaviour is stochastic and sometimes it works by chance
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(xxx)) 
    for cs in cum_steps:
        sess.run(init)
        sess.run(cs)
        print(sess.run(xxx)) 

[0, 1, 2, 3, 4]
[0, 0, 2, 3, 4]
[0, 0, 0, 3, 4]
[0, 0, 0, 0, 4]
[0, 0, 0, 0, 3]


Here is another way to write the same thing. It also does not work

In [161]:
with tf.control_dependencies([steps[0]]):
    with tf.control_dependencies([steps[1]]):
        with tf.control_dependencies([steps[2]]):
            all_in_one = tf.identity(steps[3], "all_in_one")

In [162]:
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(xxx)) 
    sess.run(all_in_one)
    print(sess.run(xxx)) 

[0, 1, 2, 3, 4]
[0, 0, 1, 2, 2]


## Solution
It seems that the identity operation is not enough to triger the dependencies.

In [163]:
prev_action = tf.identity(1, "no_action")
cum_steps2 = []
for i, (a, b) in enumerate(zip(xxx[:-1], xxx[1:])):
    with tf.control_dependencies([prev_action]):
        cs = tf.assign(b, a, name = "shift_{}_with_deps".format(i))
        cum_steps2.append(cs)
        prev_action = cs
        


In [167]:
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(xxx)) 
    for cs in cum_steps2:
        sess.run(init)
        sess.run(cs)
        print(sess.run(xxx)) 

[0, 1, 2, 3, 4]
[0, 0, 2, 3, 4]
[0, 0, 0, 3, 4]
[0, 0, 0, 0, 4]
[0, 0, 0, 0, 0]


In [179]:
prev_action = tf.identity(1, "no_action")
cum_steps3 = []
for i, (a, b) in enumerate(zip(xxx[:-1], xxx[1:])):
    with tf.control_dependencies([prev_action]):
        cs = tf.group(
            tf.assign(b, a, name = "shift_{}_with_deps".format(i))
        )
        cum_steps3.append(cs)
        prev_action = cs
        


In [184]:
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(xxx)) 
    for cs in cum_steps3:
        sess.run(init)
        sess.run(cs)
        print(sess.run(xxx)) 

[0, 1, 2, 3, 4]
[0, 0, 2, 3, 4]
[0, 0, 0, 3, 4]
[0, 0, 0, 0, 4]
[0, 0, 0, 0, 0]


# Problem

In [2]:
import tensorflow as tf

## Let's have a variable `x`
x = tf.Variable(1)
## and a fixed function `f` of `x`
f = x + 10
# Assume, we know `f` only as a tensorflow object. We do not know the formula.

## Define another variable
fx = tf.Variable(0)

## We want the following two operations executed in that order
change_x = tf.assign(x, 3)
update_fx = tf.assign(fx, f)

## Auxilliary stuff:
init = tf.global_variables_initializer()
def print_vals():
    print(sess.run({"x": x, "f": f, "fx":fx}))

##
print("This is what we want to put into one operation:")
with tf.Session() as sess:
    sess.run(init)
    print_vals()
    sess.run(change_x)
    print_vals()
    sess.run(update_fx)
    print_vals()


##
print("This does not produce good result (run several times, since the behaviour is stochastic):")
with tf.control_dependencies([change_x]):
    change_and_update = tf.assign(fx, f)

for i in range(10):
    with tf.Session() as sess:
        sess.run(init)
        sess.run(change_and_update)
        print_vals()


##
print("However this works as one would expect(run several times, since the behaviour can be stochastic):")

with tf.control_dependencies([change_x]):
    change_and_update_2 = tf.assign(fx, f + 1 - 1)

for i in range(10):
    with tf.Session() as sess:
        sess.run(init)
        sess.run(change_and_update_2)
        print_vals()
    

This is what we want to put into one operation:
{'x': 1, 'f': 11, 'fx': 0}
{'x': 3, 'f': 13, 'fx': 0}
{'x': 3, 'f': 13, 'fx': 13}
This does not produce good result (run several times, since the behaviour is stochastic):
{'x': 3, 'f': 13, 'fx': 11}
{'x': 3, 'f': 13, 'fx': 11}
{'x': 3, 'f': 13, 'fx': 11}
{'x': 3, 'f': 13, 'fx': 11}
{'x': 3, 'f': 13, 'fx': 13}
{'x': 3, 'f': 13, 'fx': 11}
{'x': 3, 'f': 13, 'fx': 11}
{'x': 3, 'f': 13, 'fx': 11}
{'x': 3, 'f': 13, 'fx': 11}
{'x': 3, 'f': 13, 'fx': 11}
However this works as one would expect(run several times, since the behaviour can be stochastic):
{'x': 3, 'f': 13, 'fx': 13}
{'x': 3, 'f': 13, 'fx': 13}
{'x': 3, 'f': 13, 'fx': 13}
{'x': 3, 'f': 13, 'fx': 13}
{'x': 3, 'f': 13, 'fx': 13}
{'x': 3, 'f': 13, 'fx': 13}
{'x': 3, 'f': 13, 'fx': 13}
{'x': 3, 'f': 13, 'fx': 13}
{'x': 3, 'f': 13, 'fx': 13}
{'x': 3, 'f': 13, 'fx': 13}


In [357]:
import tensorflow as tf

## Let's have a variable `x`
x = tf.contrib.eager.Variable(1)
## and a fixed function `f` of `x`
f = x + 10
# Assume, we know `f` only as a tensorflow object. We do not know the formula.

## Define another variable
fx = tf.contrib.eager.Variable(0)

## We want the following two operations executed in that order
change_x = tf.assign(x, 3)
update_fx = tf.assign(fx, f)

## Auxilliary stuff:
init = tf.global_variables_initializer()
def print_vals():
    print(sess.run({"x": x, "f": f, "fx":fx}))

##
print("This is what we want to put into one operation:")
with tf.Session() as sess:
    sess.run(init)
    print_vals()
    sess.run(change_x)
    print_vals()
    sess.run(update_fx)
    print_vals()


##
print("This does not produce good result (run several times, since the behaviour is stochastic):")
with tf.control_dependencies([change_x]):
    change_and_update = tf.assign(fx, f)

for i in range(10):
    with tf.Session() as sess:
        sess.run(init)
        sess.run(change_and_update)
        print_vals()


##
print("However this works as one would expect(run several times, since the behaviour can be stochastic):")

with tf.control_dependencies([change_x]):
    change_and_update_2 = tf.assign(fx, f + 1 - 1)

for i in range(10):
    with tf.Session() as sess:
        sess.run(init)
        sess.run(change_and_update_2)
        print_vals()
    

This is what we want to put into one operation:
{'fx': 0, 'x': 1, 'f': 11}
{'fx': 0, 'x': 3, 'f': 13}
{'fx': 13, 'x': 3, 'f': 13}
This does not produce good result (run several times, since the behaviour is stochastic):
{'fx': 13, 'x': 3, 'f': 13}
{'fx': 13, 'x': 3, 'f': 13}
{'fx': 13, 'x': 3, 'f': 13}
{'fx': 13, 'x': 3, 'f': 13}
{'fx': 13, 'x': 3, 'f': 13}
{'fx': 13, 'x': 3, 'f': 13}
{'fx': 13, 'x': 3, 'f': 13}
{'fx': 13, 'x': 3, 'f': 13}
{'fx': 13, 'x': 3, 'f': 13}
{'fx': 13, 'x': 3, 'f': 13}
However this works as one would expect(run several times, since the behaviour can be stochastic):
{'fx': 13, 'x': 3, 'f': 13}
{'fx': 13, 'x': 3, 'f': 13}
{'fx': 13, 'x': 3, 'f': 13}
{'fx': 11, 'x': 3, 'f': 13}
{'fx': 11, 'x': 3, 'f': 13}
{'fx': 13, 'x': 3, 'f': 13}
{'fx': 13, 'x': 3, 'f': 13}
{'fx': 13, 'x': 3, 'f': 13}
{'fx': 13, 'x': 3, 'f': 13}
{'fx': 13, 'x': 3, 'f': 13}


In [362]:
import tensorflow as tf
vscope = tf.get_variable_scope()
vscope.set_use_resource(True)

## Let's have a variable `x`
x = tf.Variable(1)
## and a fixed function `f` of `x`
f = x + 10
# Assume, we know `f` only as a tensorflow object. We do not know the formula.

## Define another variable
fx = tf.Variable(0)

## We want the following two operations executed in that order
change_x = tf.assign(x, 3)
update_fx = tf.assign(fx, f)

## Auxilliary stuff:
init = tf.global_variables_initializer()
def print_vals():
    print(sess.run({"x": x, "f": f, "fx":fx}))

##
print("This is what we want to put into one operation:")
with tf.Session() as sess:
    sess.run(init)
    print_vals()
    sess.run(change_x)
    print_vals()
    sess.run(update_fx)
    print_vals()


##
print("This works (run several times, since the behaviour is stochastic):")
with tf.control_dependencies([change_x]):
    change_and_update = tf.assign(fx, f)

for i in range(10):
    with tf.Session() as sess:
        sess.run(init)
        sess.run(change_and_update)
        print_vals()



    

This is what we want to put into one operation:
{'fx': 0, 'x': 1, 'f': 11}
{'fx': 0, 'x': 3, 'f': 13}
{'fx': 13, 'x': 3, 'f': 13}
This does not produce good result (run several times, since the behaviour is stochastic):
{'fx': 13, 'x': 3, 'f': 13}
{'fx': 11, 'x': 3, 'f': 13}
{'fx': 11, 'x': 3, 'f': 13}
{'fx': 11, 'x': 3, 'f': 13}
{'fx': 11, 'x': 3, 'f': 13}
{'fx': 11, 'x': 3, 'f': 13}
{'fx': 11, 'x': 3, 'f': 13}
{'fx': 11, 'x': 3, 'f': 13}
{'fx': 13, 'x': 3, 'f': 13}
{'fx': 11, 'x': 3, 'f': 13}
However this works as one would expect(run several times, since the behaviour can be stochastic):
{'fx': 11, 'x': 3, 'f': 13}
{'fx': 13, 'x': 3, 'f': 13}
{'fx': 11, 'x': 3, 'f': 13}
{'fx': 11, 'x': 3, 'f': 13}
{'fx': 11, 'x': 3, 'f': 13}
{'fx': 11, 'x': 3, 'f': 13}
{'fx': 11, 'x': 3, 'f': 13}
{'fx': 11, 'x': 3, 'f': 13}
{'fx': 11, 'x': 3, 'f': 13}
{'fx': 13, 'x': 3, 'f': 13}


# Problem 2

In [4]:
import tensorflow as tf

## Let's have a variable `x`
x = tf.Variable(1)
y = tf.Variable(1)
## and a fixed function `f` of `x`
f = x + 10
# Assume, we know `f` only as a tensorflow object. We do not know the formula.

## Define another variable
fx = tf.Variable(0)

## We want the following two operations executed in that order
change_y = tf.assign(y, 3)
update_x_from_y = tf.assign(x, y)
update_fx = tf.assign(fx, f)

## Auxilliary stuff:
init = tf.global_variables_initializer()
def print_vals():
    print(sess.run({"y": y, "x": x,  "f": f, "fx":fx}))

##
print("This is what we want to put into one operation:")
with tf.Session() as sess:
    sess.run(init)
    print_vals()
    sess.run(change_y)
    print_vals()
    sess.run(update_x_from_y)
    print_vals()
    sess.run(update_fx)
    print_vals()


##
print("This does not produce good result (run several times, since the behaviour is stochastic):")
with tf.control_dependencies([change_y]):
    change_y_x = tf.assign(x, y)

with tf.control_dependencies([change_y_x]):
    change_y_x_fx = tf.assign(fx, f)

for i in range(10):
    with tf.Session() as sess:
        sess.run(init)
        sess.run(change_y_x_fx)
        print_vals()


##
print("However this works as one would expect(run several times, since the behaviour can be stochastic):")
with tf.control_dependencies([change_y]):
    change_y_x = tf.assign(x, y +1 - 1)

with tf.control_dependencies([change_y_x]):
    change_y_x_fx = tf.assign(fx, f + 1 - 1)

for i in range(10):
    with tf.Session() as sess:
        sess.run(init)
        sess.run(change_y_x_fx)
        print_vals()


This is what we want to put into one operation:
{'y': 1, 'x': 1, 'f': 11, 'fx': 0}
{'y': 3, 'x': 1, 'f': 11, 'fx': 0}
{'y': 3, 'x': 3, 'f': 13, 'fx': 0}
{'y': 3, 'x': 3, 'f': 13, 'fx': 13}
This does not produce good result (run several times, since the behaviour is stochastic):
{'y': 3, 'x': 3, 'f': 13, 'fx': 13}
{'y': 3, 'x': 3, 'f': 13, 'fx': 13}
{'y': 3, 'x': 3, 'f': 13, 'fx': 11}
{'y': 3, 'x': 3, 'f': 13, 'fx': 13}
{'y': 3, 'x': 3, 'f': 13, 'fx': 13}
{'y': 3, 'x': 3, 'f': 13, 'fx': 13}
{'y': 3, 'x': 3, 'f': 13, 'fx': 11}
{'y': 3, 'x': 3, 'f': 13, 'fx': 13}
{'y': 3, 'x': 3, 'f': 13, 'fx': 11}
{'y': 3, 'x': 3, 'f': 13, 'fx': 13}
However this works as one would expect(run several times, since the behaviour can be stochastic):
{'y': 3, 'x': 3, 'f': 13, 'fx': 13}
{'y': 3, 'x': 3, 'f': 13, 'fx': 13}
{'y': 3, 'x': 3, 'f': 13, 'fx': 13}
{'y': 3, 'x': 3, 'f': 13, 'fx': 13}
{'y': 3, 'x': 3, 'f': 13, 'fx': 13}
{'y': 3, 'x': 3, 'f': 13, 'fx': 13}
{'y': 3, 'x': 3, 'f': 13, 'fx': 13}
{'y': 3, 'x